<a href="https://colab.research.google.com/github/brianpenrod/Algorithmic-Market-Recon/blob/main/market_recon_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# --- STEP 1: SYSTEM PREP (Run once) ---
try:
    import yfinance
except ImportError:
    print("Installing required libraries...")
    !pip install yfinance pandas numpy -q --upgrade

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
import warnings

# --- CONFIGURATION & PROTOCOLS ---
TICKER = "NQ=F"
OVN_START_HOUR = 18  # 6:00 PM ET
RTH_START_HOUR = 9   # 9:30 AM ET
RTH_START_MINUTE = 30
warnings.simplefilter(action='ignore', category=FutureWarning)

def clean_float(val):
    """Helper to sanitize numpy/pandas scalar values."""
    try:
        if isinstance(val, (pd.Series, np.ndarray, list)):
            return float(val.iloc[0]) if hasattr(val, 'iloc') else float(val[0])
        return float(val)
    except:
        return 0.0

def get_market_intelligence(ticker_symbol):
    print(f"--- INITIATING FULL SPECTRUM SCAN FOR {ticker_symbol} ---")

    # 1. ROBUST DATA FETCH
    try:
        # Fetch Intraday (5m) for Regime & Momentum
        intraday = yf.download(ticker_symbol, period="5d", interval="5m", progress=False)
        # Fetch Daily (1y) for Volatility & Strategic Trend
        daily = yf.download(ticker_symbol, period="1y", interval="1d", progress=False)
    except Exception as e:
        print(f"DATA FETCH ERROR: {e}")
        return None, None, None

    # 2. DATA VALIDATION
    if intraday is None or intraday.empty or daily is None or daily.empty:
        return None, None, None

    # 3. NORMALIZE COLUMNS (Fix for yfinance v0.2+ MultiIndex issues)
    if isinstance(intraday.columns, pd.MultiIndex):
        intraday.columns = intraday.columns.get_level_values(0)
    if isinstance(daily.columns, pd.MultiIndex):
        daily.columns = daily.columns.get_level_values(0)

    # 4. TIMEZONE ALIGNMENT (US/Eastern)
    try:
        if intraday.index.tz is None:
            intraday.index = intraday.index.tz_localize('UTC').tz_convert('US/Eastern')
        else:
            intraday.index = intraday.index.tz_convert('US/Eastern')
    except Exception as e:
        print(f"TIMEZONE ERROR: {e}")
        return None, None, None

    # 5. DEFINE OVERNIGHT SESSION
    now_et = datetime.now(pytz.timezone('US/Eastern'))
    today_date = now_et.date()
    yesterday_date = today_date - timedelta(days=1)

    ovn_start = now_et.replace(year=yesterday_date.year, month=yesterday_date.month, day=yesterday_date.day,
                               hour=OVN_START_HOUR, minute=0, second=0, microsecond=0)
    ovn_end = now_et.replace(year=today_date.year, month=today_date.month, day=today_date.day,
                             hour=RTH_START_HOUR, minute=RTH_START_MINUTE, second=0, microsecond=0)

    ovn_data = intraday[(intraday.index >= ovn_start) & (intraday.index < ovn_end)]

    # 6. CALCULATE METRICS
    metrics = {}

    # REGIME: Overnight High/Low/VWAP
    if not ovn_data.empty:
        metrics['ON_High'] = ovn_data['High'].max()
        metrics['ON_Low'] = ovn_data['Low'].min()
        metrics['ON_Range'] = metrics['ON_High'] - metrics['ON_Low']

        # VWAP
        v = ovn_data['Volume']
        tp = (ovn_data['High'] + ovn_data['Low'] + ovn_data['Close']) / 3
        metrics['ON_VWAP'] = (tp * v).cumsum() / v.cumsum()
        metrics['ON_VWAP_Final'] = metrics['ON_VWAP'].iloc[-1]
    else:
        metrics.update({'ON_High': 0.0, 'ON_Low': 0.0, 'ON_Range': 0.0, 'ON_VWAP_Final': 0.0})

    # VOLATILITY: 14-Day ATR
    high_low = daily['High'] - daily['Low']
    high_close = np.abs(daily['High'] - daily['Close'].shift())
    low_close = np.abs(daily['Low'] - daily['Close'].shift())
    tr_df = pd.concat([high_low, high_close, low_close], axis=1)
    metrics['ATR_14'] = tr_df.max(axis=1).rolling(window=14).mean().iloc[-1]

    # MOMENTUM: 14-Period RSI (on 5m chart)
    delta = intraday['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    metrics['RSI_14'] = 100 - (100 / (1 + rs)).iloc[-1]

    # TREND: 200-Day SMA
    metrics['SMA_200_Daily'] = daily['Close'].rolling(window=200).mean().iloc[-1]

    current_price = intraday['Close'].iloc[-1]

    return metrics, current_price, today_date

def run_mission():
    """Main Execution Block"""
    try:
        stats, price, report_date = get_market_intelligence(TICKER)

        if stats is None:
            print("MISSION ABORTED: Data stream invalid.")
            return

        # Sanitize Inputs
        cur_price = clean_float(price)
        on_high = clean_float(stats.get('ON_High', 0))
        on_low = clean_float(stats.get('ON_Low', 0))
        on_vwap = clean_float(stats.get('ON_VWAP_Final', 0))
        on_range = clean_float(stats.get('ON_Range', 0))
        daily_atr = clean_float(stats.get('ATR_14', 1)) # Prevent div/0
        rsi_val = clean_float(stats.get('RSI_14', 50))
        sma_200 = clean_float(stats.get('SMA_200_Daily', 0))

        # Logic: Volatility Usage
        vol_usage = (on_range / daily_atr) * 100 if daily_atr > 0 else 0

        # Logic: Directional Bias
        trend_bias = "BULLISH" if cur_price > sma_200 else "BEARISH"

        # Logic: Momentum State
        if rsi_val > 70: mom_state = "OVERBOUGHT (Risk High)"
        elif rsi_val < 30: mom_state = "OVERSOLD (Bounce Risk)"
        elif rsi_val > 50: mom_state = "POSITIVE"
        else: mom_state = "NEGATIVE"

        # Logic: Final Regime
        if vol_usage < 40:
            regime_state = "COMPRESSED (Breakout Imminent)"
        elif vol_usage > 100:
            regime_state = "EXTENDED (Mean Reversion)"
        else:
            regime_state = "NORMAL FLOW"

        # --- REPORT GENERATION ---
        print("\n" + "="*45)
        print(f"  QUANT SITREP: {report_date}")
        print("="*45)
        print(f"ASSET CLASS    : {TICKER}")
        print(f"CURRENT PRICE  : {cur_price:,.2f}")
        print("-" * 45)
        print(f"OVERNIGHT HIGH : {on_high:,.2f}  [Resistance]")
        print(f"OVERNIGHT LOW  : {on_low:,.2f}  [Support]")
        print(f"OVERNIGHT VWAP : {on_vwap:,.2f}  [Equilibrium]")
        print("-" * 45)
        print("VOLATILITY INTELLIGENCE (ATR-14):")
        print(f"EXP. DAILY MOVE: {daily_atr:.2f} pts")
        print(f"CURRENT RANGE  : {on_range:.2f} pts ({vol_usage:.1f}%)")
        print("-" * 45)
        print("TECHNICAL DRIVERS:")
        print(f"RSI (5m Momentum): {rsi_val:.2f}")
        print(f"SMA (200 Daily)  : {sma_200:.2f}")
        print(f"STRATEGIC BIAS   : {trend_bias} (Price vs 200 SMA)")
        print(f"TACTICAL MOMENTUM: {mom_state}")
        print("-" * 45)
        print("FINAL REGIME ANALYSIS:")
        print(f">> STATE: {regime_state}")
        print(f">> PLAN : Wait for {trend_bias} confirmation outside ON Range.")
        print("="*45)

    except Exception as e:
        print(f"RUNTIME ERROR: {e}")
        import traceback
        traceback.print_exc()

# --- FIRE MISSION ---
if __name__ == "__main__":
    run_mission()


--- INITIATING FULL SPECTRUM SCAN FOR NQ=F ---

  QUANT SITREP: 2025-12-10
ASSET CLASS    : NQ=F
CURRENT PRICE  : 25,661.00
---------------------------------------------
OVERNIGHT HIGH : 25,745.75  [Resistance]
OVERNIGHT LOW  : 25,616.00  [Support]
OVERNIGHT VWAP : 25,678.37  [Equilibrium]
---------------------------------------------
VOLATILITY INTELLIGENCE (ATR-14):
EXP. DAILY MOVE: 412.41 pts
CURRENT RANGE  : 129.75 pts (31.5%)
---------------------------------------------
TECHNICAL DRIVERS:
RSI (5m Momentum): 54.66
SMA (200 Daily)  : 22666.63
STRATEGIC BIAS   : BULLISH (Price vs 200 SMA)
TACTICAL MOMENTUM: POSITIVE
---------------------------------------------
FINAL REGIME ANALYSIS:
>> STATE: COMPRESSED (Breakout Imminent)
>> PLAN : Wait for BULLISH confirmation outside ON Range.
